### Calculate distance between all offshore wind areas

In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [2]:
df_nodes = pd.read_csv("../Output/nodes.csv")
df_nodes["geometry"] = df_nodes["geometry"].apply(wkt.loads)
nodes = gpd.GeoDataFrame(df_nodes, crs="EPSG:4326")
nodes = nodes.to_crs("EPSG:25833")
nodes

,Node,Longitude,Latitude,geometry
0,Austria,14.430000,47.460000,POINT (457038.809 5256441.760)
1,Bosnia H,17.810000,44.060000,POINT (725065.181 4882376.507)
2,Belgium,4.600000,50.720000,POINT (-233330.788 5670463.302)
3,Bulgaria,25.250000,42.710000,POINT (1339724.705 4779845.144)
4,Switzerland,8.150000,46.820000,POINT (-22441.596 5207987.953)
...,...,...,...,...
64,Nordavind C,20.019911,71.728123,POINT (675483.230 7965912.714)
65,Vestavind E,3.853030,59.109692,POINT (-136458.754 6605653.720)
66,Sørvest D,3.840226,56.464644,POINT (-185922.125 6313761.564)
67,Energyhub North,1.440000,58.420000,POINT (-288723.459 6555561.818)


In [3]:
results = None
for row in nodes.itertuples():
    other_nodes = nodes[nodes.index < row.Index]
    distances = other_nodes.geometry.distance(row.geometry)
    distances_nodes = other_nodes.copy()
    distances_nodes["distance"] = distances / 1000 # m to km
    for name, value in row._asdict().items():
        distances_nodes[f"to_{name}"] = value

    if results is None:
        results = distances_nodes
    else:
        results = pd.concat([results, distances_nodes])

results

,Node,Longitude,Latitude,geometry,distance,to_Index,to_Node,to_Longitude,to_Latitude,to_geometry
0,Austria,14.430000,47.460000,POINT (457038.809 5256441.760),460.177084,1,Bosnia H,17.81,44.06,POINT (725065.1806372444 4882376.506848703)
0,Austria,14.430000,47.460000,POINT (457038.809 5256441.760),804.999390,2,Belgium,4.60,50.72,POINT (-233330.7880281218 5670463.301955454)
1,Bosnia H,17.810000,44.060000,POINT (725065.181 4882376.507),1240.807652,2,Belgium,4.60,50.72,POINT (-233330.7880281218 5670463.301955454)
0,Austria,14.430000,47.460000,POINT (457038.809 5256441.760),1003.134450,3,Bulgaria,25.25,42.71,POINT (1339724.7053934564 4779845.144450622)
1,Bosnia H,17.810000,44.060000,POINT (725065.181 4882376.507),623.152479,3,Bulgaria,25.25,42.71,POINT (1339724.7053934564 4779845.144450622)
...,...,...,...,...,...,...,...,...,...,...
63,Nordavind D,18.808962,71.450117,POINT (635164.986 7931860.745),1869.151340,68,Energyhub Central,3.29,56.05,POINT (-227445.80113339517 6273660.105354942)
64,Nordavind C,20.019911,71.728123,POINT (675483.230 7965912.714),1918.071877,68,Energyhub Central,3.29,56.05,POINT (-227445.80113339517 6273660.105354942)
65,Vestavind E,3.853030,59.109692,POINT (-136458.754 6605653.720),344.235969,68,Energyhub Central,3.29,56.05,POINT (-227445.80113339517 6273660.105354942)
66,Sørvest D,3.840226,56.464644,POINT (-185922.125 6313761.564),57.726447,68,Energyhub Central,3.29,56.05,POINT (-227445.80113339517 6273660.105354942)


In [4]:
results = results[["Node", "to_Node", "distance"]]
results

,Node,to_Node,distance
0,Austria,Bosnia H,460.177084
0,Austria,Belgium,804.999390
1,Bosnia H,Belgium,1240.807652
0,Austria,Bulgaria,1003.134450
1,Bosnia H,Bulgaria,623.152479
...,...,...,...
63,Nordavind D,Energyhub Central,1869.151340
64,Nordavind C,Energyhub Central,1918.071877
65,Vestavind E,Energyhub Central,344.235969
66,Sørvest D,Energyhub Central,57.726447


Export CSV

In [5]:
results.to_csv("../Output/dist_all_areas.csv", index=False, encoding='utf-8-sig')